# Ideal QKD Implementation with EVE

This implementation will make the assumption that Eve is not present. The cells below contain the photon and qubit modules. Normally we would save these as python files and import them using statements, similar to how we have used statements like "import random" or "import math". Due to the constraints of using Colab, we are going to store them in hidden cells. Take a look at the classes. What are the methods? What are the members? Once you are done, put "#@title" at the top of each cell and press play. Click on the whitespace to the right. If you did it right, the cell should collapse and you should only see "SHOW CODE" in the cell.





In [81]:
import random
import numpy as np

class InputError(Exception):
    def __int__(self, expression, message):
        self.expression = expression
        self.message = message

class Photon:

    def __init__(self, Hcomp=0, Vcomp=0):
        self.alpha = Hcomp
        self.beta  = Vcomp

    # This is for debugging purposes only!
    def toString(self):
        if np.isreal(self.alpha):
            string = str(self.alpha) + "|H> "
        else:
            string = str(self.alpha) + "|H> "
        if np.isreal(self.beta):
            if self.beta >= 0:
                string += "+ " + str(self.beta) + "|V>"
            else:
                string += "- " + str(-self.beta) + "|V>"
        else:
            string += "+ " + str(self.beta) + "|V>"
        return string

    def prepareVacuum(self):
        energyPerMode = 0.5; # in units of hbar*omega
        x0 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y0 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        x1 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y1 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        self.alpha = complex(x0, y0)
        self.beta  = complex(x1, y1)

    def prepare(self, alpha, beta, avgPhotonNumber):
        if avgPhotonNumber < 0:
            raise InputError()
        vac = Photon()
        vac.prepareVacuum()
        self.alpha = alpha * np.sqrt(avgPhotonNumber) + vac.alpha
        self.beta  = beta  * np.sqrt(avgPhotonNumber) + vac.beta

    def prepareH(self, avgPhotonNumber):
        self.prepare(1, 0, avgPhotonNumber)

    def prepareV(self, avgPhotonNumber):
        self.prepare(0,1, avgPhotonNumber)

    def prepareD(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2),  1/np.sqrt(2), avgPhotonNumber)

    def prepareA(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2), -1/np.sqrt(2), avgPhotonNumber)

    

    def measureDA(self, probDarkCount):
        a = self.alpha
        b = self.beta
        self.alpha = (a+b)/np.sqrt(2)
        self.beta  = (a-b)/np.sqrt(2)
        outcome = self.measureHV(probDarkCount)
        if outcome == "H": return "D"
        if outcome == "V": return "A"
        else: return outcome

    def measureHV(self, probDarkCount):
        if probDarkCount < 0 or probDarkCount > 1:
            raise InputError
        threshold  = -0.5*np.log(1 - np.sqrt(1-probDarkCount))
        intensityH = abs(self.alpha)**2
        intensityV = abs(self.beta)**2
        # The photon is absorbed by the detector:
        self.prepareVacuum()
        # The outcome is determined by threshold exceedances:
        if intensityH <= threshold and intensityV <= threshold:
            return "N" # no detection (invalid measurement)
        elif intensityH >  threshold and intensityV <= threshold:
            return "H" # single H photon detected
        elif intensityH <=  threshold and intensityV > threshold:
            return "V" # single V photon detected
        else:
            return "M" # multiple detections (invalid measurement)

# Alice creates a message

The length of the message will determine how long the key, and what number of qubits they will need to use. 


In [82]:
alice_message = "hello"   #put any message you like here, try to keep it small though!
#You will need 8 bits per character.

#this parameter will become more important in the next phase of your project. It gives you 
# some leeway in case some of the key gets intercepted. You can ignore this for now.


wiggle_room = 100


# Alice generates the raw key

In [83]:


n = 8*len(alice_message) + wiggle_room # number of qubits. We multiply by 8 because of ASCII

# Alice --------------------------------------------


keyAlice = ""
for i in range(n): # Iterate over the number of qubits.
    # Append a random character ('0' or '1') to the end.
    keyAlice += random.choice(['0','1'])
print("keyAlice    = " + keyAlice)


keyAlice    = 01011001110001100010100110000001111101110101111100011011101110001010101101001110001001111100010010101011101110010010001101110100110010111110


# Alice chooses the encoding basis for each key bit.
 This should be a randomly chosen string of '+'s and 'x's with '+'=H/V, 'x'=D/A. You do not need to create a function for this unless you want to. A simple "for loop" will be fine.




In [84]:
basisAlice = ""
# TODO: Put your code here.
for i in range(n):
    basisAlice += random.choice(['+','x'])
print("basisAlice  = " + basisAlice)


# Your output should just be a bunch of  '+' and  'x' s, with no spaces in between.

basisAlice  = +x+x+x+x+x++xxx++++++x++x+x++xxx+xxxx+++xx++xxxxxx+xx++x+xxx+xx+x+xx+x+xx+xx+xxxx+xxx++++xx+x+x+xx++xx+xx++xx+xxx++x+++x++xxx+x++xx++xxx++xx


# Alice selects a qubit state according to the key and basis.

Write a function that takes basisAlice ( string of +, x) and keyAlice (a string of 0's and 1's) as arguments and returns photonAlice, a string of polarizations  'H', 'V', 'D',  or 'A'.

In [85]:


def photonAlice_generator( your_basisAlice,your_keyAlice ):
# TODO: Put your code here.
    your_photonAlice = ""
    for i in range( len (your_basisAlice)):
        if your_basisAlice[i]=='+':
            if your_keyAlice[i]=='0':
                your_photonAlice += 'H'
            elif your_keyAlice[i]=='1':
                your_photonAlice += 'V'
        elif your_basisAlice[i]=='x':
            if your_keyAlice[i]=='0':
                your_photonAlice += 'D'
            elif your_keyAlice[i]=='1':
                your_photonAlice += 'A'
    #print("photonAlice  = " + photonAlice)
    return your_photonAlice





# Sanity check: photonAlice_generator()


In [86]:
test_basis = '+x+'

test_key = '010'

test_photon =  photonAlice_generator(test_basis, test_key)

print(test_photon)

# you should see: HAH

HAH


In [112]:
#create qubitAlice in order to be passed into the next part of the protocol

photonAlice = photonAlice_generator(basisAlice, keyAlice)

print(qubitAlice) #just to check!


HAHAVDHAVAHHDAAHHHVHVDHVAHDHHDDAVAAADVVVDAHVAAAADDHAAHVAVDAAVDDHAHADVDVADVDDVAADDHADDVVVVADHDVDHADVHADVAAHVAAHDADHVDHHVAHVAADVDHVADHVDAAVVAD


# Alice prepares and sends each qubit.
Create a function that includes the photonArray initialization listed below. This function shoud take in photonAlice as an argument, and then use the correct method of the Photon class (.prepareH(), .prepareV(), .prepareD(), .prepareA(), etc.)

In [113]:
average_photon_number = 1 # playwith this : insert [description of parameter]

def photonArray_generator(your_photonAlice,average_photon_number):

    your_photonArray = [Photon() for i in range(len(your_photonAlice))] # Initializes an array of instances of the Photon class

    # TODO: Put your code here.
    for i in range(len(your_photonAlice)):
        if   your_photonAlice[i]=='H': your_photonArray[i].prepareH(average_photon_number)
        elif your_photonAlice[i]=='V': your_photonArray[i].prepareV(average_photon_number)
        elif your_photonAlice[i]=='D': your_photonArray[i].prepareD(average_photon_number)
        elif your_photonAlice[i]=='A': your_photonArray[i].prepareA(average_photon_number)
    return your_photonArray

    

In [114]:
photonArray = photonArray_generator(photonAlice,1)

In [89]:
random.seed(123)

test_photonAlice  = 'HAH'

test_photonArray = photonArray_generator(test_photonAlice,1)

print(len(test_photonArray))

for i in range(len(test_photonArray)):
   print(test_photonArray[i].toString())

# you should see : 
#(1.2021142166123333+0.06900569632415206j)|H> + (-0.1992923612468027+0.13137399451330123j)|V>
#(0.8205500555172319-0.08112282097832324j)|H> + (-1.1448454295134607-0.10132338482239818j)|V>
#(1.1764607134499159-0.23629937837582562j)|H> + (-0.23477638910021187+0.3846557133557717j)|V>


3
(1.2021142166123333+0.06900569632415206j)|H> + (-0.1992923612468027+0.13137399451330123j)|V>
(0.8205500555172319-0.08112282097832324j)|H> + (-1.1448454295134607-0.10132338482239818j)|V>
(1.1764607134499159-0.23629937837582562j)|H> + (-0.23477638910021187+0.3846557133557717j)|V>


# Eve   --------------------------------------------
 You should implement this section after completing Alice and Bob.Eve is allowed to do whatever she wants to the photonAlice array. She cannot, however, have knowledge of Alice's or Bob's choice of bases, nor Bob's measurement outcomes, until they are publicly announced.Eve selects a subsample of photons from Alice to measure.  "interceptIndex" should be a string of n characters. Use the convention '0'=ignored, '1'=intercepted. For ex : A string of only 0's would mean Eve ignores every photon (bad idea) and a string of 1's would mean she measures every photon (also a bad idea). It would be a very good idea (hint hint) to crreate a variable that counts the number of intercepted photons. 

In [90]:
interceptIndex = ""


# TODO: Put your code here.

# Some considerations : How are you going to choose which photons to sample? How can you do this 
# in a way that is repeatable with different values of "n ", or the total number of photons? How can you do this 
# in a way that makes it easy to vary the number of photons you sample? Feel free to experiment or 
#even write functions to create interceptIndex to your specifications

fracIntercepted = 0.2 # the probability Eve chooses to intercept a photon
numIntercepted = 0    # a running total of the number of interceptions
for i in range(n):
    if random.uniform(0,1) < fracIntercepted:
        interceptIndex += '1'
        numIntercepted += 1
    else:
        interceptIndex += '0'




# Sanity Check - interceptIndex

In [91]:

print(interceptIndex)

if len(interceptIndex)== n:
    print( "Length is correct")



#Should print : A string of 1's and 0's that is the same length as " n " or the number of photons


01010100011010000000000000000000000101000100100011010000000000010001110000011010101000000000000000010100100001000001100000000100010000000000
Length is correct


# Eve chooses a basis to measure each intercepted photon.
basisEve should be a string of n characters.
 Use the convention '0'=H/V, '1'=D/A, ' '=not measured. Try writing a function that takes interceptIndex as an argument and output basisEve.

In [92]:


# TODO: Put your code here. basisEve should be a blank string ' ', if interceptIndex is a '0' and 
# a random choice of bases if interceptIndex is a '1'.

def basisEve_generator(your_interceptIndex):
    
    basisEve = ""

    for i in range(len(your_interceptIndex)):
        if your_interceptIndex[i] == '1':
            basisEve += random.choice(['0','1'])
        else:
            basisEve += ' '
    return basisEve

                

# Sanity check - basisEve
Only run this cell if you are testing, the random.seed() line can cause you some problems down the road.


In [103]:

test_interceptIndex = '101'
random.seed(123)


test_basisEve = basisEve_generator(test_interceptIndex)

print(test_basisEve)

#You should see: '0 1'


0 1


In [94]:
#Once you have passed the sanity check run this:

basisEve = basisEve_generator( interceptIndex)

# Eve performs a measurement on each photon.
The function outcomeEve_generator() should take in photonArray, basisEve, interceptIndex and prob_dark_count as arguments and return outcomeEve. outComeEve should be appended measurements in HV or DA using methods of the Photon class, with prob_dark_count passed in as arguments. The variable "outcomeEve" should be a string of n characters.Use the convention 'H','V','D','A'; or a  ' ' if the photon is not measured.

In [137]:

def outcomeEve_generator(your_photonArray, your_basisEve, your_interceptIndex, your_prob_dark_count):
# TODO: Put your code here.

    your_outcomeEve = ""

    for i in range(len(your_interceptIndex)):
        if your_interceptIndex[i] == '1':
            if your_basisEve[i]=='0':
                your_outcomeEve += your_photonArray[i].measureHV(your_prob_dark_count)
            elif your_basisEve[i]=='1':
                your_outcomeEve += your_photonArray[i].measureDA(your_prob_dark_count)
        else:
            your_outcomeEve += ' '
            
    return  your_outcomeEve


# Sanity check: outcomeEve_generator





In [107]:
random.seed(123)

test_prob_dark_count = 0.1
test_outcomeEve =  outcomeEve_generator(test_photonArray, test_basisEve, test_interceptIndex, test_prob_dark_count)

print(test_outcomeEve)

#You should see: 'N N'

N N


In [138]:
#Run this once you've passed the sanity check

outcomeEve = outcomeEve_generator(photonArray, basisEve, interceptIndex, prob_dark_count)

# Bob   --------------------------------------------

Bob chooses a basis to measure each qubit. (This is similar to what Alice does.)

In [109]:
basisBob = ""
# TODO: Put your code here.
for i in range(n):
    basisBob += random.choice(['+','x']) # '+'=H/V, 'x'=D/A
print("basisBob    = " + basisBob)



basisBob    = +++xx+++x+x++xx++++++xxxxx+xxx+x+xxxxx+++xx+xxx+xxx+++x+x+x+xx+x++xxx+++x++xxxxx+++x++x+x+x+xxx+xxx++x++x+++xx+xxx+x++xx++x+xxxxx++x+x+x+x+x


# Bob performs a measurement on each qubit.

In [117]:
prob_dark_count = 0.1

# Use the methods of the Qubit class to measure each qubit.
outcomeBob = ""
# TODO: Put your code here.
for i in range(n):
    if basisBob[i]=='+':
        outcomeBob += photonArray[i].measureHV(prob_dark_count)
    elif basisBob[i]=='x':
        outcomeBob += photonArray[i].measureDA(prob_dark_count)
print("outcomeBob  = " + outcomeBob)
# This should be a string of the characters 'H', 'V', 'D', 'A'.

outcomeBob  = HHHNDHNHNNNHVNANNNVNNNNNNNNNNNNANNANDDNNNANNNNANDDNHNNNNDVNNNDHANNNDNVVHDNMNANADVNNDNVDNNNNNDNDHADNNHDVNNHVHNNHNNNVNHNAAHVNNNNNANHHANNHNVMNN


In [123]:
# Bob infers the raw key.
keyBob = ""
for i in range(n):
    if outcomeBob[i] == 'H' or outcomeBob[i] == 'D':
        keyBob += '0'
    elif outcomeBob[i] == 'V' or outcomeBob[i] == 'A':
        keyBob += '1'
    elif outcomeBob[i] == 'N' or outcomeBob[i] == 'M':
        keyBob += '-'

print(keyBob)

000-00-0---01-1---1------------1--1-00---1----1-00-0----01---001---0-1100---1-101--0-10-----0-0010--001--010--0---1-0-1101-----1-001--0-1---


# -----------------------------------------------------------
# Alice and Bob now publicly announce which bases they chose.
# -----------------------------------------------------------

In [124]:
# Alice and Bob extract their sifted keys.
siftedAlice = ""
siftedBob   = ""
# TODO: Put your code here.
for i in range(n):
    if basisAlice[i] == basisBob[i]:
        siftedAlice += keyAlice[i]
        siftedBob   += keyBob[i]
    else:
        siftedAlice += ' '
        siftedBob   += ' '
print("siftedAlice = " + siftedAlice)
print("siftedBob   = " + siftedBob)

siftedAlice = 0 01  0    0 110001010  1    0 111110 11 1 1111 00   0    1  0   010  1 01 0 110 0 0 1 1  000 0010 0 01 101 1  10 1000 1011 0 0     10 11  0
siftedBob   = 0 0-  -    0 -1---1---  -    - 1--1-0 -- 1 ---1 00   -    -  0   --0  1 0- - -10 - 0 1 -  --0 0010 - 01 -01 -  -- 1-0- 101- - -     -- -1  -


# Compare Alice and Bob's sifted keys.


In [125]:
numMatch = 0
if len(siftedAlice) != len(siftedBob):
    print("Sifted keys are different lengths!")
else:
    for i in range(len(siftedAlice)):
        if siftedAlice[i] == siftedBob[i]:
           numMatch += 1
    matchPercent = numMatch / len(siftedAlice) * 100
print(str(matchPercent) + "% match")

68.57142857142857% match


# New to this implementation - Alice and Bob select a portion of their sifted keys to sample.

In [158]:

# sampleIndex should be a string of n characters.
# Use the convention '0'=ignored, '1'=sampled
sampleIndex = ""

fracSampled = 0.1
numSampled = 0

for i in range(len(siftedAlice)):
    if siftedAlice[i] != ' ' and random.uniform(0, 1) < fracSampled:
        sampleIndex += "1"
        numSampled += 1
    else:
        sampleIndex += "0"
print("sampleIndex = " + sampleIndex)

sampleIndex = 00000000000101000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000001000011100000000000000000


In [159]:
# Alice and Bob use their sample to estimate the quantum bit error rate (QBER).
# The QBER is the fraction of mismatches within the sampled portion.
# For large samples, it should be close to the actual QBER,
# which Alice and Bob, of course, do not know.
sampledBobQBER = 0
numSampled = 0
numMatch = 0
for i in range(len(sampleIndex)):
    if sampleIndex[i] == '1':
        numSampled += 1
    if (sampleIndex[i] == '1') and (siftedAlice[i] == siftedBob[i]):
        numMatch += 1
if numSampled > 0:
    sampledBobQBER = 1 - numMatch / numSampled
else:
    sampledBobQBER = float('nan')
    
    

In [188]:
#len(sampleIndex)
print(len(siftedBob))
print(len(sampleIndex))

print( "siftedAlice " + siftedAlice)
print("siftedBob " + siftedBob)

140
140
siftedAlice 0 01  0    0 110001010  1    0 111110 11 1 1111 00   0    1  0   010  1 01 0 110 0 0 1 1  000 0010 0 01 101 1  10 1000 1011 0 0     10 11  0
siftedBob 0 0-  -    0 -1---1---  -    - 1--1-0 -- 1 ---1 00   -    -  0   --0  1 0- - -10 - 0 1 -  --0 0010 - 01 -01 -  -- 1-0- 101- - -     -- -1  -


In [193]:
# Alice and Bob remove the portion of their sifted keys that was sampled.
# Since a portion of the sifted key was publicly revealed, it cannot be used.
# secureAlice and secureBob should be strings of length n.
# Use the convention '0', '1', ' '=removed
secureAlice = ""
secureBob = ""

secure_Bob_count = 0

for i in range(len(sampleIndex)):
    if (sampleIndex[i] == '1'):
        secureAlice += " "
    elif (sampleIndex[i] == '0') and siftedAlice[i] == '0':
        secureAlice += '0'
    elif (sampleIndex[i] == '0') and siftedAlice[i] == '1':
        secureAlice += '1'
    elif siftedAlice[i] == ' ':
        secureAlice += " "

print(secureAlice)
print(len(secureAlice))


for i in range(len(sampleIndex)):
    if (sampleIndex[i] == '1'):
        secureBob += " "
    elif (sampleIndex[i] == '0') and siftedBob[i] == '0':
        secureBob += '0'

    elif (sampleIndex[i] == '0') and siftedBob[i] == '1':
        secureBob += '1'

    elif siftedBob[i] == ' ':
        
        secureBob += " "
    else:
        secureBob += " "
                

0 01  0       10001010  1    0 111110 11 1 1111 0    0    1  0   010  1 01 0 110 0 0 1 1  000 0010 0 01 101 1  10 1 00 1    0 0     10 11  0
140


In [194]:
print("SI     " + sampleIndex[0:10])
print("SB     " + siftedBob[0:10])
print("secBob " +  secureBob[0:10])

SI     0000000000
SB     0 0-  -   
secBob 0 0       


In [161]:
# Alice and Bob make a hard determination whether the channel is secure.
# TODO: Choose a good safety threshold.
safetyThreshold = 0.15  # change this value!

if sampledBobQBER > safetyThreshold:
    channelSecure = False  # something's fishy - hit the kill switch!
else:
    channelSecure = True  # everything looks fine!

print(channelSecure)

False


# Eve ------------------------------------------------------------------


In [162]:
# Below is the method Eve uses to extract her stolen key.
# You need not change any of what follows.

# Eve infers the raw key.
# keyEve should be a string of n characters.
# Use the convention '0', '1', '-'=invalid measurement, ' '=not measured
keyEve = ""
for i in range(len(interceptIndex)):
    if interceptIndex[i] == '1':
        if outcomeEve[i] == 'H' or outcomeEve[i] == 'D':
            keyEve += '0'
        elif outcomeEve[i] == 'V' or outcomeEve[i] == 'A':
            keyEve += '1'
        else:
            keyEve += '-'  # invalid measurement
    else:
        keyEve += ' '
        
print (keyEve)        


 - - -   -- -                      - 1   1  -   1- -           -   ---     -- - - 0                - -  -    -     --        -   -          


In [195]:
print(len(interceptIndex))
print(len(keyEve))
print(len(keyBob))
print(len(basisAlice))
print(len(basisBob))
print(len(basisEve))

140
140
140
140
140
140


In [164]:
# Eve extracts her sifted key.
# Knowing what Alice and Bob have publicly revealed, Eve
# now selects which portion of her sifted key to keep.
# stolenEve should be strings of length n.
# Use the convention '0', '1', ' '=removed
stolenEve = ""
for i in range(len(interceptIndex)):
    if interceptIndex[i] == '1' and keyEve[i] != '-' and keyBob[i] != '-' and basisEve[i] == basisAlice[i] and basisEve[
        i] == basisBob[i]:
        stolenEve += keyEve[i]
    else:
        stolenEve += ' '
        
print(stolenEve)        

# ANALYSIS -------------------------------------------------------------


In [196]:
print(len(secureAlice))
print(len(secureBob))

print(secureAlice)
print(secureBob)

140
140
0 01  0       10001010  1    0 111110 11 1 1111 0    0    1  0   010  1 01 0 110 0 0 1 1  000 0010 0 01 101 1  10 1 00 1    0 0     10 11  0
0 0           1   1            1  1 0    1    1 0            0     0  1 0     10   0 1      0 0010   01  01       1 0  1                1   


In [171]:
# Below is a standard set of metrics to evaluate each protocol.
# You need not change any of what follows.

# Compare Alice and Bob's sifted keys.
numMatchBob = 0
actualBobQBER = 0
secureKeyRateBob = 0
secureKeyLengthBob = 0
for i in range(len(secureAlice)):
    if secureAlice[i] != ' ':
        secureKeyLengthBob += 1
        if secureAlice[i] == secureBob[i]:
            numMatchBob += 1

# Compute the actual quantum bit error rate for Bob.
if secureKeyLengthBob > 0:
    actualBobQBER = 1 - numMatchBob / secureKeyLengthBob
else:
    actualBobQBER = float('nan')
# Compute the secure key rate, assuming each trial takes 1 microsecond.
secureKeyRateBob = (1 - actualBobQBER) * secureKeyLengthBob / n * 1e6;

IndexError: string index out of range

In [172]:
# Compute the actual quantum bit error rate for Bob.
if secureKeyLengthBob > 0:
    actualBobQBER = 1 - numMatchBob / secureKeyLengthBob
else:
    actualBobQBER = float('nan')
# Compute the secure key rate, assuming each trial takes 1 microsecond.
secureKeyRateBob = (1 - actualBobQBER) * secureKeyLengthBob / n * 1e6;

# Compare Alice and Eve's sifted keys.
numMatchEve = 0
actualEveQBER = 0
stolenKeyRateEve = 0
stolenKeyLengthEve = 0
for i in range(len(stolenEve)):
    if stolenEve[i] != ' ':
        stolenKeyLengthEve += 1
        if secureAlice[i] == stolenEve[i]:
            numMatchEve += 1

In [173]:
# Compute the actual quantum bit error rate for Eve.
if stolenKeyLengthEve > 0:
    actualEveQBER = 1 - numMatchEve / stolenKeyLengthEve
else:
    actualEveQBER = float('nan')
# Compute the stolen key rate, assuming each trial takes 1 microsecond.
stolenKeyRateEve = (1 - actualEveQBER) * stolenKeyLengthEve / n * 1e6;


# DISPLAY RESULTS ------------------------------------------------------

In [174]:
print("")
print("keyAlice     = " + keyAlice)
print("basisAlice   = " + basisAlice)
print("photonAlice  = " + photonAlice)
print("basisEve     = " + basisEve)
print("outcomeEve   = " + outcomeEve)
print("keyEve       = " + keyEve)
print("basisBob     = " + basisBob)
print("outcomeBob   = " + outcomeBob)
print("keyBob       = " + keyBob)
print("")
print("siftedAlice  = " + siftedAlice)
print("siftedBob    = " + siftedBob)
print("")
print("secureAlice  = " + secureAlice)
print("secureBob    = " + secureBob)
print("stolenEve    = " + stolenEve)
print("")
if not channelSecure:
    secureKeyRateBob = 0;
    stolenKeyRateEve = 0;
    print("*********************************************")
    print("* ALERT! The quantum channel is not secure. *")
    print("*********************************************")
    print("")
print("sampledBobQBER = " + str(sampledBobQBER))
print("actualBobQBER  = " + str(actualBobQBER))
print("actualEveQBER  = " + str(actualEveQBER))
print("")
print("secureKeyRateBob = " + str(secureKeyRateBob / 1000) + " kbps")
print("stolenKeyRateEve = " + str(stolenKeyRateEve / 1000) + " kbps")



keyAlice     = 01011001110001100010100110000001111101110101111100011011101110001010101101001110001001111100010010101011101110010010001101110100110010111110
basisAlice   = +x+x+x+x+x++xxx++++++x++x+x++xxx+xxxx+++xx++xxxxxx+xx++x+xxx+xx+x+xx+x+xx+xx+xxxx+xxx++++xx+x+x+xx++xx+xx++xx+xxx++x+++x++xxx+x++xx++xxx++xx
photonAlice  = HAHAVDHAVAHHDAAHHHVHVDHVAHDHHDDAVAAADVVVDAHVAAAADDHAAHVAVDAAVDDHAHADVDVADVDDVAADDHADDVVVVADHDVDHADVHADVAAHVAAHDADHVDHHVAHVAADVDHVADHVDAAVVAD
basisEve     =  1 1 0   00 1                      1 0   0  0   10 1           0   011     00 0 0 0                0 1  1    1     11        0   1          
outcomeEve   =  N N N   NN N                      N V   V  N   AN N           N   NNN     NM N N H                N N  N    N     NN        N   N          
keyEve       =  - - -   -- -                      - 1   1  -   1- -           -   ---     -- - - 0                - -  -    -     --        -   -          
basisBob     = +++xx+++x+x++xx++++++xxxxx+xxx+x+xxxxx+++xx+xxx+